# Day 7 - Lab 2: Building Agent Frontends with the A2A Framework

**Objective:** Use the A2A (Ask-Advise-Act) Framework to build a guided, conversational user interface for an AI agent.

**Estimated Time:** 135 minutes

**Introduction:**
A powerful agent is only effective if users can interact with it easily. Often, users know their goal but not how to phrase it for an AI. The A2A Framework solves this by creating a guided dialogue. In this lab, you will build a Streamlit application that uses A2A to help a non-technical user generate a complex SQL query by breaking the process down into three steps: Ask, Advise, and Act.

## Step 1: Setup

We will need to install the `a2a` and `streamlit` libraries. Note that this lab requires you to create and run a standalone Python script (`.py`), as Streamlit applications are not run inside Jupyter notebooks.

In [1]:
import sys
import os

# Add the project's root directory to the Python path
try:
    # This works when running as a script
    project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
except NameError:
    # This works when running in an interactive environment (like a notebook)
    # We go up two levels from the notebook's directory to the project root.
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
# This helper will install packages if they are not found
import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        %pip install -q {package}

install_if_missing('a2a')
install_if_missing('streamlit')
install_if_missing('langchain_openai')

from utils import setup_llm_client, save_artifact
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

a2a not found, installing...


  DEPRECATION: Building 'a2a' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'a2a'. Discussion can be found at https://github.com/pypa/pip/issues/6334


Note: you may need to restart the kernel to use updated packages.
✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Problem Statement

Your task is to build a Streamlit app called "SQL Query Generator". A user will provide a high-level goal (e.g., "show me all our customers in California") and a database schema. The app will use the A2A framework to guide the user to a valid SQL query.

## Step 3: Your Task

**Task:** Create a Python script named `sql_query_generator.py` that implements the A2A flow.

**Instructions:**
1.  **Setup the UI:** Create a Streamlit UI with a text area for the user's goal and another for the database schema.
2.  **Ask Phase:** When the user clicks "Generate Query", your app should first enter the **Ask** phase. It should ask the LLM to generate 1-2 clarifying questions based on the user's goal and schema (e.g., "Should the results be ordered by a specific column?"). Display these questions to the user with text input boxes for their answers.
3.  **Advise Phase:** Once the user answers the clarifying questions, enter the **Advise** phase. Send the original goal, the schema, the clarifying questions, and the user's answers to the LLM. Ask the LLM to create a step-by-step "plan" for how it will construct the query.
4.  **Act Phase:** Finally, enter the **Act** phase. Send the plan and all previous context to the LLM and ask it to generate the final SQL query.
5.  Display the final SQL query to the user in a code block.

In [ ]:
# TODO: Create a file named 'sql_query_generator.py' and add the following code.
# You will need to complete the prompts for each phase of the A2A flow.

sql_app_code = """
import streamlit as st
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-4o", api_key=os.getenv("OPENAI_API_KEY"))

st.title("SQL Query Generator with A2A")

schema = st.text_area("Database Schema", height=150, placeholder="CREATE TABLE customers (...)")
goal = st.text_area("What is your goal?", placeholder="Show me all customers in California ordered by last name.")

# Initialize session state variables
if 'step' not in st.session_state:
    st.session_state.step = "START"

if st.button("Generate Query"):
    if schema and goal:
        st.session_state.step = "ASK"
        # --- ASK PHASE ---
        ask_prompt = f''' # Your Ask Phase Prompt Here '''
        response = llm.invoke(ask_prompt).content
        st.session_state.questions = [q.strip() for q in response.split('?') if q.strip()]
        st.session_state.answers = ["" for _ in st.session_state.questions]
    else:
        st.error("Please provide both a schema and a goal.")

if st.session_state.step == "ASK" and 'questions' in st.session_state:
    st.subheader("Ask: Clarifying Questions")
    for i, q in enumerate(st.session_state.questions):
        st.session_state.answers[i] = st.text_input(f"{q}?", key=f"ans{i}")
    
    if st.button("Submit Answers"):
        st.session_state.step = "ADVISE"
        # --- ADVISE PHASE ---
        advise_prompt = f''' # Your Advise Phase Prompt Here '''
        st.session_state.plan = llm.invoke(advise_prompt).content
        st.experimental_rerun()

if st.session_state.step == "ADVISE" and 'plan' in st.session_state:
    st.subheader("Advise: Query Plan")
    st.markdown(st.session_state.plan)
    if st.button("Looks Good, Generate SQL"):
        st.session_state.step = "ACT"
        # --- ACT PHASE ---
        act_prompt = f''' # Your Act Phase Prompt Here '''
        query = llm.invoke(act_prompt).content
        if '```' in query:
            query = query.split('```')[1].lstrip('sql').strip()
        st.session_state.query = query
        st.experimental_rerun()

if st.session_state.step == "ACT" and 'query' in st.session_state:
    st.subheader("Act: Final SQL Query")
    st.code(st.session_state.query, language='sql')
"""

from utils import save_artifact
save_artifact(sql_app_code, "labs/Day_07_Advanced_Agent_Workflows/sql_query_generator.py")
print("Saved 'sql_query_generator.py'. To run it, open your terminal and execute: streamlit run labs/Day_07_Advanced_Agent_Workflows/sql_query_generator.py")

## Lab Conclusion

Congratulations! You have built a practical application using the A2A framework to create a guided, conversational experience. This pattern is incredibly powerful for turning complex tasks (like writing SQL) into a simple, step-by-step process for users. You've learned how to break down a problem into the Ask, Advise, and Act phases to create more user-friendly and effective AI tools.